In [1]:
import pathlib
import subprocess
PATH_DIRECTORY = pathlib.Path().resolve()
DATA_DIRECTORY = PATH_DIRECTORY / "Data"
subprocess.call('./Data/get_data.sh', shell=True)

./Data/get_data.sh: line 7: [: /home/warcry98/Data/DataEngineer/Data_Engineering_ZoomCamp-Homeworks/Week1/SQL/Data/taxi+_zone_lookup.csv: binary operator expected


0

In [2]:
# Load green tripdata
import pandas as pd
tripdata_df = pd.read_csv(DATA_DIRECTORY / "green_tripdata_2019-01.csv.gz", compression='gzip')
tripdata_df.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2,1,NaN
5,2,2019-01-01 00:12:35,2019-01-01 00:19:09,N,1,49,17,1,1.05,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,1,1,NaN
6,2,2019-01-01 00:47:55,2019-01-01 01:00:01,N,1,255,33,1,3.77,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,1,1,NaN
7,1,2019-01-01 00:12:47,2019-01-01 00:30:50,N,1,76,225,1,4.10,16.0,0.5,0.5,0.00,0.0,NaN,0.3,17.30,1,1,NaN
8,2,2019-01-01 00:16:23,2019-01-01 00:39:46,N,1,25,89,1,7.75,25.5,0.5,0.5,0.00,0.0,NaN,0.3,26.80,1,1,NaN
9,2,2019-01-01 00:58:02,2019-01-01 01:19:02,N,1,85,39,1,3.68,15.5,0.5,0.5,0.00,0.0,NaN,0.3,16.80,1,1,NaN


In [3]:
# transform lpep_pickup_datetime and lpep_dropoff_datetime to datetime type
tripdata_df['lpep_pickup_datetime'] = pd.to_datetime(tripdata_df['lpep_pickup_datetime'])
tripdata_df['lpep_dropoff_datetime'] = pd.to_datetime(tripdata_df['lpep_dropoff_datetime'])
print(tripdata_df.dtypes)

VendorID                          int64
lpep_pickup_datetime     datetime64[ns]
lpep_dropoff_datetime    datetime64[ns]
store_and_fwd_flag               object
RatecodeID                        int64
PULocationID                      int64
DOLocationID                      int64
passenger_count                   int64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                      int64
trip_type                         int64
congestion_surcharge            float64
dtype: object


In [4]:
# Load taxi zone
zone_df = pd.read_csv(DATA_DIRECTORY / "taxi+_zone_lookup.csv", delimiter=',')
zone_df.head(10)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


In [5]:
# Replacement dictionary to map pandas dtypes to SQL dtypes
replacement = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64[ns]': 'timestamp',
    'timedelta64[ns]': 'varchar',
}

col_tripdata = ", ".join("{} {}"
              .format(n, d) for (n ,d)
              in zip(tripdata_df.columns, tripdata_df.dtypes.replace(replacement)))

col_zone = ", ".join("{} {}"
          .format(n, d) for (n, d)
          in zip(zone_df.columns, zone_df.dtypes.replace(replacement)))

print(col_tripdata)

VendorID int, lpep_pickup_datetime timestamp, lpep_dropoff_datetime timestamp, store_and_fwd_flag varchar, RatecodeID int, PULocationID int, DOLocationID int, passenger_count int, trip_distance float, fare_amount float, extra float, mta_tax float, tip_amount float, tolls_amount float, ehail_fee float, improvement_surcharge float, total_amount float, payment_type int, trip_type int, congestion_surcharge float


In [6]:
# Save tripdata df to csv
tripdata_df.to_csv(DATA_DIRECTORY / "tripdata.csv",
                   header=tripdata_df.columns,
                   index=False, encoding='utf-8')

# Check csv created
check = pd.read_csv(DATA_DIRECTORY / "tripdata.csv")
check.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2,1,NaN
5,2,2019-01-01 00:12:35,2019-01-01 00:19:09,N,1,49,17,1,1.05,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,1,1,NaN
6,2,2019-01-01 00:47:55,2019-01-01 01:00:01,N,1,255,33,1,3.77,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,1,1,NaN
7,1,2019-01-01 00:12:47,2019-01-01 00:30:50,N,1,76,225,1,4.10,16.0,0.5,0.5,0.00,0.0,NaN,0.3,17.30,1,1,NaN
8,2,2019-01-01 00:16:23,2019-01-01 00:39:46,N,1,25,89,1,7.75,25.5,0.5,0.5,0.00,0.0,NaN,0.3,26.80,1,1,NaN
9,2,2019-01-01 00:58:02,2019-01-01 01:19:02,N,1,85,39,1,3.68,15.5,0.5,0.5,0.00,0.0,NaN,0.3,16.80,1,1,NaN


In [7]:
import psycopg2

host = 'localhost'
default_db = 'postgres'
user = 'postgres'
password = ''

dbname = 'zoomcamp_week1'
tbname_trip = 'green_tripdata'
tbname_zone = 'taxi_zone'

In [8]:
def create_database():
    conn = None
    try:
        conn = psycopg2.connect(
            host=host,
            dbname=default_db,
            user=user,
            password=password
        )

        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute("CREATE DATABASE %s;" % dbname)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if conn is not None:
            conn.close()
        print("PotsgreSQL connection is closed")

In [9]:
def create_tables():
    conn = None
    try:
        conn_str = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
        with psycopg2.connect(conn_str) as conn:
            with conn.cursor() as cur:
                
                # Create table trip data
                cur.execute("CREATE TABLE %s (%s);" % (tbname_trip, col_tripdata))
                print("Table tripdata created successfully")

                # Open csv tripdata
                tripdata_file = open(DATA_DIRECTORY / "tripdata.csv")
                
                # SQL Copy tripdata to db
                SQL = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','" % tbname_trip
                cur.copy_expert(sql=SQL, file=tripdata_file)

                # Create table taxi zone
                cur.execute("CREATE TABLE %s (%s);" % (tbname_zone, col_zone))
                print("Table taxi zone created successfully")

                # # Open csv taxi zone
                zone_file = open(DATA_DIRECTORY / "taxi+_zone_lookup.csv")

                # # SQL Copy taxi zone to db
                SQL = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','" % tbname_zone
                cur.copy_expert(sql=SQL, file=zone_file)
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if conn is not None:
            conn.close()
        print("PostgreSQL connection is closed")

In [10]:
def drop_tables():
    conn = None
    try:
        conn_str = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
        with psycopg2.connect(conn_str) as conn:
            with conn.cursor() as cur:
                
                # Drop table trip data
                cur.execute("DROP TABLE IF EXISTS %s;" % tbname_trip)

                # Drop table taxi zone
                cur.execute("DROP TABLE IF EXISTS %s;" % tbname_zone)
                
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if conn is not None:
            conn.close()
        print("PostgreSQL connection is closed")

In [11]:
create_database()
drop_tables()
create_tables()

database "zoomcamp_week1" already exists

PotsgreSQL connection is closed
PostgreSQL connection is closed
Table tripdata created successfully
Table taxi zone created successfully
PostgreSQL connection is closed
